# Stoneburner, Kurt
- ## DSC 530 - Week 11

In [ ]:
####################################################
####################################################
####################################################
####//*** Reused Functions from previous exercises
####################################################
####################################################
####################################################
# //*****************************************
# //*** Build a probability mass function
# //*****************************************
# //*** Returns Series as a PMF
# //*****************************************
def build_pmf(input_series):
    output_series = input_series.copy()
    total_values = input_series.sum()
    for value,freq in output_series.items():
        #print(f"{value} {freq} {total_values} {freq/total_values}")
        output_series.loc[value] = freq/total_values
    return output_series

# //*** Build a Cumulative Distribution Function from a Probability Mass Function
# //*** Returns a Series
def build_cdf(input_series):
    # //*** If input is not panda or pd series, try to convert it
    if not isinstance(input_series,pd.core.series.Series):
        input_series = pd.Series(input_series)
        
    # //*** If input is np.Array
    output_series = input_series.copy()
    cumulative_value = 0
    for value,freq in output_series.items():
        #print(f"{value} {freq} {cumulative_value} {freq + cumulative_value}")
        cumulative_value = freq + cumulative_value
        output_series.loc[value] = cumulative_value
    return output_series

# //*** Retrieve a percentile value from a CDF.
# //*** Returns index value closest to input parameter percentile.
def get_cdf_percentile(input_cdf,percentile):
    #print(f"{input_cdf}")
    #//*** Initialize output to first value
    output = input_cdf.index[0]
    
    #//*** Loop through all items till the value exceeds the percentile
    #//*** Return value from last loop
    for index,value in input_cdf.items():
        
        if value > percentile:
            return output
        else:
            output = index
####################################################
####################################################
####################################################
####//*** END Functions from previous exercises
####################################################
####################################################
####################################################

In [ ]:
####################################################
####################################################
####################################################
####//*** Thinkstats function
####################################################
####################################################
####################################################
def CleanData(resp):
    """Cleans respondent data.

    resp: DataFrame
    """
    resp.cmdivorcx.replace([9998, 9999], np.nan, inplace=True)

    resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)
    resp['duration'] = (resp.cmdivorcx - resp.cmmarrhx) / 12.0
    resp['durationsofar'] = (resp.cmintvw - resp.cmmarrhx) / 12.0

    month0 = pd.to_datetime('1899-12-15')
    dates = [month0 + pd.DateOffset(months=cm) 
             for cm in resp.cmbirth]
    resp['decade'] = (pd.DatetimeIndex(dates).year - 1900) // 10
    
    resp.cmmarrhx.replace([9997, 9998, 9999], np.nan, inplace=True)
    resp['agemarry'] = (resp.cmmarrhx - resp.cmbirth) / 12.0
    resp['age'] = (resp.cmintvw - resp.cmbirth) / 12.0

In [ ]:
####################################################
####################################################
####################################################
####//*** New Functions for this week
####################################################
####################################################
####################################################
# def build_hazard_function(sf):
#-------- Takes a survival function and returns a hazard Function
###################################################################
# build_kaplan_meier_hazard_curve(input_complete, input_ongoing):
#--------- Generates an estimates the longevity of a 
#--------- population and returns a hazard function 
###################################################################
# build_survival_from_hazard(input_hazard_series):
#--------- Builds Survival Function from a hazard function 
###################################################################
#//*** Build hazard function from a cdf
#//*** Hazard Function: (1-CDF(x)) - (1-CDF(x+1)) / (1-CDF(x))
#//*** Difference between a value and the next value (ie the difference in two values) divided by the first value
#//*** This is a percentage of difference between sequential survivor function (1-CDF[x]) values
#//**** Returns a dictionary of hazard values
def build_hazard_function(sf):
    #//*** Convert cdf to survival function
    #sf = 1 - pd.Series(input_cdf)
    
    #//*** output dictionary
    out_dict= {}
    
    for index,value in enumerate(sf):
        #//*** Skip the last value since it generates an out of array error.
        #//*** Had troubles parsing a series len(-1) so did it this way
        if index < len(sf)-1:
            if sf.iloc[index] != 0:
                #//*** Value = Hazard Value
                out_dict[value] = ( sf.iloc[index] - sf.iloc[index+1] / sf.iloc[index] )
            else:
                out_dict[value] = 0
    return out_dict

#//*** Build a Kaplan Meier Survival Curve to estimate a survival function
#//*** For an estimated/predicted lifetime

def build_kaplan_meier_hazard_curve(input_complete, input_ongoing):
    #//*** Build Histogram Dictionary of Complete and ongoing
    #//*** No reason to import a Counter Library for Basic stuff
    #//*** Usual histogram method of value_counts and sort_index, combined with a for items loop
    #//*** to convert to a dictionary
    #from collections import Counter    
    #hist_complete_counter = Counter(input_complete)
    #hist_ongoing_counter =  Counter(input_ongoing)
    hist_combined_list = []    

    hist_complete = {}
    for index, value in input_complete.value_counts().sort_index().items():
        #//*** Add Unique value to the combined list
        if index not in hist_complete.keys():
            hist_combined_list.append(index)

        #//*** Add to Dictionary
        hist_complete[index] = value

    hist_ongoing = {}
    for index, value in input_ongoing.value_counts().sort_index().items():
        #//*** Add Unique value to the combined list
        if index not in hist_complete.keys():
            hist_combined_list.append(index)

        #//*** Add to Dictionary
        hist_ongoing[index] = value

    #hist_combined_list = list(hist_complete_counter | hist_ongoing_counter)
    hist_combined_list.sort()
    
    at_risk = len(input_complete) + len(input_ongoing)
    

    #//*** Create an empty Survival curve using the hist_comibined_list as a key
    #//*** All values will be represented in the Survival Curve.
    #//*** If something is missed it will be represented as a NaN
    survival_curve = pd.Series(index=hist_combined_list,dtype='float')

    #//*** Go through each value in the ongoing and complete dictionaries
    #//*** Get the count of each, or zero if not found
    for x in hist_combined_list:
        

        #ended = hist_complete_counter[x]
        #censored = hist_ongoing_counter[x]
        
        if x in hist_complete.keys():
            ended = hist_complete[x]
        else:
            ended = 0

        if x in hist_ongoing.keys():
            censored = hist_ongoing[x]
        else:
            censored = 0

        #//*** Calculate the percentage of ended vs the remaining at risk
        survival_curve[x] = ended / at_risk

        
        #//*** Reduced the at_risk total by the totals found
        at_risk -= ended + censored

    return survival_curve

def build_survival_from_hazard(input_hazard_series):
    #//****************************************************
    #//**** Convert Hazard Function to Survival Function
    #//****************************************************
    #ts = input_hazard_series.index
    #//*** cumprod - cumulative product. Essentially builds a CDF from hazard curve
    #//**** (1-hazard_curve).cumprod() Survival Function (reciprocal CDF) of hazard curve
    sf = (1 - input_hazard_series).cumprod()
    #//****************************************************
    
    return (pd.Series(index=input_hazard_series.index, data=sf))

    

In [ ]:
# //****************************************************************************************
# //*** Set Working Directory to thinkstats folder.
# //*** This pseudo-relative path call should work on all Stoneburner localized projects. 
# //****************************************************************************************

import os
import sys
workingPath = os.getcwd().replace("coding", "ThinkStats2\\code")
sys.path.insert(1, workingPath)
os.chdir(workingPath)

In [ ]:
# //*** Imports and Load Data
import nsfg
import thinkstats2
import thinkplot
import first
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random


**Exercise:**    In NSFG Cycles 6 and 7, the variable `cmdivorcx` contains the date of divorce for the respondent’s first marriage, if applicable, encoded in century-months.

Compute the duration of marriages that have ended in divorce, and the duration, so far, of marriages that are ongoing. Estimate the hazard and survival curve for the duration of marriage.

Use resampling to take into account sampling weights, and plot data from several resamples to visualize sampling error.

Consider dividing the respondents into groups by decade of birth, and possibly by age at first marriage.

In [ ]:
#//****************************************
#//*** Prepare, Clean and load the data
#//****************************************

#//*** Import data sets
resp6 = nsfg.ReadFemResp(dct_file='2002FemResp.dct',dat_file='2002FemResp.dat.gz')
resp7 = nsfg.ReadFemResp(dct_file='2006_2010_FemRespSetup.dct',dat_file='2006_2010_FemResp.dat.gz')

#//*** Clean Data using included Thinkstsats code
CleanData(resp6)
CleanData(resp7)

#//*** Combine 2002 and 2010 datasets
resp = pd.concat([resp6,resp7], sort=False)#resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)

#//*** There are a number of coding errors with durationsofar. A few marriages and divorces are listed 
#//*** are negative lengths. Drop these rows
resp.drop(index=resp[ resp['durationsofar'] <= 0 ].index, inplace=True)
resp.drop(index=resp[ resp['duration'] <= 0 ].index, inplace=True)

#//*** complete is a list of marriage lengths for those that have been married
complete = round(resp[resp.evrmarry==1]['duration'].dropna(),1)

#//*** ongoing is a list of those never married. This list is filled with nan
ongoing =  round( resp[resp.evrmarry==0]['durationsofar'],1)



In [ ]:
print(resp['decade'].unique())

In [ ]:
#//*** Estimate the projected hazard curve of marriage lengths of the projected lifetimes using
#//*** kaplan meier method. Retuns Hazard Function
hazard_curve = build_kaplan_meier_hazard_curve(complete,ongoing)

#//*** Convert Kaplan Meier Hazard Function to survival function
sf = build_survival_from_hazard(hazard_curve)

#//*** Perform weighted resampling to estimate the errors
#//*** Convert Survival function to CDF. Use the CDF for weighted resampling of complete. 
#//*** Reference: https://www.python-course.eu/weighted_choice_and_sample.php
sf_cdf = 1-sf

import random

complete_len = len(complete)
resampled_survival_curves = []
ci_dict = {}
#//*** Generate 100 weighted samples
for _ in range(100):
    
    #//*** Generate random percentages with the range of the CDF.
    complete_weight = [ random.uniform(sf_cdf.min(),sf_cdf.max()) for __ in range(complete_len) ]
    
    #//*** Convert List of Random weights into values from the CDF.
    complete_resample = pd.Series( data=[ get_cdf_percentile(sf_cdf,__) for __ in complete_weight ] )
    #//*** Ongoing is empty with NaN
    ongoing_resample = pd.Series( data=[ np.nan for __ in range(len(ongoing)) ] )
    
    #//*** Build Estimated Lifetime Hazard Function from resampled/random data
    resample_hazard_curve = build_kaplan_meier_hazard_curve(complete_resample,ongoing_resample)
    
    #//*** Convert Hazard Function to Survival Function
    re_sample_ts = resample_hazard_curve.index
    re_sample_ss = (1 - resample_hazard_curve).cumprod()
    loop_series = pd.Series(index=re_sample_ts, data=re_sample_ss)
    resampled_survival_curves.append(loop_series)
    
    #//*** Collect Values for each index over all resamples
    for x in loop_series.index:
        if x not in ci_dict.keys():
            ci_dict[x] = [loop_series[x]]
        else:
            ci_dict[x].append(loop_series[x])


In [ ]:
#//*** Get resampled Hi/Low (error) for each value. This represents the resampled error for each value
min_vals = []
max_vals = []
for x in ts:
    if x in ci_dict.keys():
        min_vals.append(pd.Series(ci_dict[x]).min() )
        max_vals.append(pd.Series(ci_dict[x]).max() )

#//*** Missing one value, add it in from the main data
min_vals.append(ss.iloc[-1])
max_vals.append(ss.iloc[-1])

plt.plot(ts,ss,1,"r")
#plt.plot(ts, min_vals, ts, max_vals, color='black')
plt.fill_between(ts, min_vals, max_vals,color='gray')
plt.plot(ts,ss,1,"r")
plt.show()

In [ ]:
plt.plot(ts,ss,1,"b")
plt.plot(ts, 1-ss,1,"r")

plt.show()



plt.plot(hazard_curve.index,hazard_curve,1,"b")
plt.ylim(hazard_curve.min(),hazard_curve.max())
plt.show()

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE